# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
import os
import joblib
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
import requests
import json
import sklearn
from azureml.automl.runtime.onnx_convert import OnnxConverter
import pandas as pd
from pandas import DataFrame

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Heart-Disease-Predection'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RBDDCWTNF to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
#Creating Compute
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
#Create compute cluster


# choose a name for your cluster
cluster_name= "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws , name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target ...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2' , max_nodes=4)
    #create the cluster
    cpu_cluster=ComputeTarget.create(ws,cluster_name,compute_config)
    print('cpu-cluster has been created')


Creating a new compute target ...
cpu-cluster has been created


In [4]:
#Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Heart Failure Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 
        print("Found")

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/BMUHAMMDI/nd00333-capstone/master/starter_file/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key)


df = dataset.to_pandas_dataframe()
df.describe()

Found


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00
mean,60.83,0.43,581.84,0.42,38.08,0.35,263358.03,1.39,136.63,0.65,0.32,130.26,0.32
std,11.89,0.50,970.29,0.49,11.83,0.48,97804.24,1.03,4.41,0.48,0.47,77.61,0.47
min,40.00,0.00,23.00,0.00,14.00,0.00,25100.00,0.50,113.00,0.00,0.00,4.00,0.00
25%,51.00,0.00,116.50,0.00,30.00,0.00,212500.00,0.90,134.00,0.00,0.00,73.00,0.00
50%,60.00,0.00,250.00,0.00,38.00,0.00,262000.00,1.10,137.00,1.00,0.00,115.00,0.00
75%,70.00,1.00,582.00,1.00,45.00,1.00,303500.00,1.40,140.00,1.00,1.00,203.00,1.00
max,95.00,1.00,7861.00,1.00,80.00,1.00,850000.00,9.40,148.00,1.00,1.00,285.00,1.00


In [5]:
from sklearn.model_selection import train_test_split

training_data, test_data = train_test_split(df)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory
if not os.path.isdir("data"):
    os.mkdir("data")

pd.DataFrame(training_data).to_csv("data/train_data.csv", index=False)
ds=ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='HeartDataset', overwrite=True,show_progress=True)
train_data=TabularDatasetFactory.from_delimited_files(path=ds.path('HeartDataset/train_data.csv'))
print(train_data.to_pandas_dataframe())

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files
      age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0   75.00        0                       582         0                 20   
1   72.00        0                       364         1                 20   
2   60.00        1                        96         1                 60   
3   50.00        0                       196         0                 45   
4   45.00        1                       130         0                 35   
..    ...      ...                       ...       ...                ...   
219 51.00        1                       582         1                 35   
220 70.00        1                        75         0                 35   
221 68.00        1                       220         0                 35   
222 70.00        0                        69         0                 40   

In [8]:
# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 25,
                    "max_concurrent_iterations":4,
                    "iterations": 20,                     
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(  
    task='classification',
    compute_target='cpu-cluster',
    primary_metric = 'accuracy',
    training_data= train_data,
    label_column_name ='DEATH_EVENT',
    enable_onnx_compatible_models=True,
    featurization='auto',
    enable_early_stopping= True,
    **automl_settings
)

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_0f6b24f8-c742-4dec-8fb8-606c34a7e53a

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [11]:
best_automl_run,fitted_model = remote_run.get_output()
print(best_automl_run)
print(fitted_model)

Run(Experiment: Heart-Disease-Predection,
Id: AutoML_0f6b24f8-c742-4dec-8fb8-606c34a7e53a_19,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                min_child_weight=0,
                                                                                                min_split_gain=0.42105263157894735,
      

In [12]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
Heart-Disease-Predection,AutoML_0f6b24f8-c742-4dec-8fb8-606c34a7e53a_19,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [13]:
best_automl_run.get_metrics()

{'precision_score_micro': 0.9021739130434782,
 'balanced_accuracy': 0.8612471637839285,
 'average_precision_score_micro': 0.9381393483274991,
 'AUC_weighted': 0.9136935405651985,
 'weighted_accuracy': 0.9281093611117506,
 'AUC_micro': 0.9345775594057084,
 'recall_score_macro': 0.8612471637839285,
 'recall_score_weighted': 0.9021739130434782,
 'precision_score_weighted': 0.9079351540907605,
 'AUC_macro': 0.9136935405651982,
 'matthews_correlation': 0.758838088161985,
 'norm_macro_recall': 0.722494327567857,
 'recall_score_micro': 0.9021739130434782,
 'accuracy': 0.9021739130434782,
 'average_precision_score_macro': 0.9099145117143029,
 'precision_score_macro': 0.9033207216759847,
 'average_precision_score_weighted': 0.9329526651243603,
 'f1_score_macro': 0.8688648981246432,
 'log_loss': 0.3681620550791397,
 'f1_score_weighted': 0.8965596381638361,
 'f1_score_micro': 0.9021739130434782,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_0f6b24f8-c742-4dec-8fb8-606c34a7e53a_

In [30]:
best_automl_run.tags

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":2,"CurrentNodeCount":4}',
 'ensembled_iterations': '[2, 18, 4, 3, 8, 15, 16]',
 'ensembled_algorithms': "['RandomForest', 'LightGBM', 'RandomForest', 'RandomForest', 'RandomForest', 'LightGBM', 'LightGBM']",
 'ensemble_weights': '[0.125, 0.125, 0.125, 0.25, 0.125, 0.125, 0.125]',
 'best_individual_pipeline_score': '0.8798418972332016',
 'best_individual_iteration': '2',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

In [15]:
#TODO: Save the best model
best_automl_run,onnx_model =remote_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model,file_path="./automl_model.onnx")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
#Register the best automl model,
automl_model =remote_run.register_model(model_name = 'automl_best_model')

In [17]:
script_file_name ='inference/sscore.py'
best_automl_run.download_file('outputs/scoring_file_v_1_0_0.py','inference/sscore.py')

In [18]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_automl_run.get_environment())

In [31]:
#Local Deployment
from azureml.core.webservice import LocalWebservice
local_config = LocalWebservice.deploy_configuration(port=9001)
local_service = Model.deploy(ws, "test", [automl_model], inference_config, local_config)
local_service.wait_for_deployment(show_output=True)

#Set up the deployment_config as webservice
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, enable_app_insights=True)

#Deploy the model
service = Model.deploy(
    workspace = ws,
    name = "heart-failure-model",
    models = [automl_model],
    inference_config = inference_config,
    deployment_config = aci_config, overwrite=True)

#wait until deployment is complete
service.wait_for_deployment(show_output = True)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry viennaglobal.azurecr.io
Logging into Docker registry viennaglobal.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM viennaglobal.azurecr.io/azureml/azureml_4f3cee89203e005745d1830c04fe722a
 ---> c354c32ff2d1
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 84682dfe70b6
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImY5ZDVhMDg1LTU0ZGMtNDIxNS05YmE2LWRhZDVkODZlNjBhMCIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzNDc2NyIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzNDc2NyIsIndvcmtzcGFjZUlkIjoiNWQxNWJiYzAtNmRmYy00ODg4LWFjZWQtNGU0OGM2N2RkZjk0In0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 851ebd0d5871
 ---> cb05ac954e09
Step 4/5 : RUN mv '/var/azureml-app/tmpc70l7slt.py' /var/azureml-app/main.py
 ---> Running in 2d9394cf1fee
 ---> 3075d4b8a9d6
Step 5/5 : 

In [32]:
print("Scoring uri:" , service.scoring_uri)

Scoring uri: http://0534fc25-0720-4538-8e65-22b6c2ae6564.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [33]:
scoring_uri = 'http://0534fc25-0720-4538-8e65-22b6c2ae6564.southcentralus.azurecontainer.io/score'
headers = {'Content-Type':'application/json'}

input_data = json.dumps({'data':[{
    'age':75,
    'anaemia':0,
    'creatinine_phosphokinase':582,
    'diabetes':0,
    'ejection_fraction':20,
    'high_blood_pressure':1,
    'platelets':265000,
    'serum_creatinine':1.9,
    'serum_sodium':130,
    'sex':1,
    'smoking':0,
    'time':4}
    ]
        })

input_data2 = json.dumps({'data':[{
    'age':40,
    'anaemia':0,
    'creatinine_phosphokinase':321,
    'diabetes':0,
    'ejection_fraction':35,
    'high_blood_pressure':0,
    'platelets':265000,
    'serum_creatinine':1,
    'serum_sodium':130,
    'sex':1,
    'smoking':0,
    'time':198}
    ]
        })


response = requests.post(scoring_uri, data=input_data, headers=headers)

print("Result:",response.text)

Result: "{\"result\": [1]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [34]:
#Print the webservice logs
print(service.get_logs())

2021-01-15T15:24:10,073655519+00:00 - rsyslog/run 
2021-01-15T15:24:10,073877616+00:00 - gunicorn/run 
2021-01-15T15:24:10,073877716+00:00 - iot-server/run 
2021-01-15T15:24:10,161732106+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_8eff28b157f42edcd2424a5aae6c8074/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

In [35]:
#Delete the service
service.delete()